In [1]:
import yaml
import time
import os
import datetime
import warnings
import sys

warnings.simplefilter(action='ignore', category=FutureWarning)
sys.path.append("../pytorch3dunet/unet3d")
sys.path.append("../src/utils/")

import torch
import numpy as np


from torch import nn
from box import Box
from loguru import logger
from tqdm import tqdm
from torch.cuda import amp
from transformers import get_cosine_with_hard_restarts_schedule_with_warmup


import losses
from base_scripts import set_seed, create_train_test_loader, create_fold_from_data
from my_losses import MyDiceLoss, CategoricalCrossEntropyLoss
from my_metrics import print_metric_MeanIoU

from model import ResidualUNet3D

In [2]:
import os 
import pydicom
import numpy as np 
import matplotlib.pyplot as plt
import ipyvolume as ipv
import nibabel as nib

In [3]:
data = create_fold_from_data("data_for_segmentation.csv")
train_loader, valid_loader = create_train_test_loader(data=data,
                                                          test_fold=1,
                                                          size_img=(256,256,96),
                                                          path_to_folder_with_imgs="C:\\KAGGLE\\MEDICINE\\data\\train_images",
                                                          path_to_folder_with_masks="C:\\KAGGLE\\MEDICINE\\data\\segmentations_numpy",
                                                          typ_augm="v1",
                                                          batch_size=1,
                                                          num_workers=0
                                                          )


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
model = ResidualUNet3D(in_channels=1,
                                   out_channels=5,
                                   f_maps=[32,64,128,256,512],
                                   final_sigmoid=False).to(device)

model.load_state_dict(torch.load("ResidualUNet3D_fold_1_last_epochs.pt", map_location= device))
model.to(device)
device

device(type='cuda')

In [6]:
model_v2 = ResidualUNet3D(in_channels=1,
                                   out_channels=5,
                                   f_maps=[32,64,128,256,512],
                                   final_sigmoid=False).to(device)

model_v2.load_state_dict(torch.load("ResidualUNet3D_fold_1_last_epochs_v2.pt", map_location= device))
model_v2.to(device)
device

device(type='cuda')

In [7]:
import random
import torch.nn.functional as F

In [8]:
model.eval()
model_v2.eval()
device

device(type='cuda')

In [9]:
valid_iterator = iter(valid_loader)

In [73]:
batch = next(valid_iterator) 
with torch.no_grad():
    X_batch = batch[0].to(device)
    masks_batch = batch[1].to(device)
    
    pred_masks = model(X_batch).to('cpu')
    

In [74]:
with torch.no_grad():
    pred_masks_v2 = model_v2(X_batch).to('cpu')

In [75]:
pred_masks = torch.argmax(pred_masks, dim=1)[0].numpy()
pred_masks_v2 = torch.argmax(pred_masks_v2, dim=1)[0].numpy()

In [76]:
pred_masks.shape, pred_masks_v2.shape

((256, 256, 96), (256, 256, 96))

In [77]:
img  =  X_batch[0][0].to('cpu').numpy()
real_mask = masks_batch[0].to('cpu').numpy()

In [91]:
# Создайте трехмерную визуализацию
fig = ipv.figure()

# Отобразите объемные данные
# vol = ipv.volshow(np.where(real_mask == 3, 1, 0) , lighting=True)
vol = ipv.volshow(np.where(real_mask == 3, 1, 0) + np.where(pred_masks_v2 == 3, 1, 0), lighting=True)
# Добавьте интерактивность (вращение, масштабирование и т.д.)
ipv.style.box_off()
ipv.style.axes_off()
ipv.style.set_style_light()


# Покажите визуализацию
ipv.show()

Container(children=[VBox(children=(HBox(children=(Label(value='levels:'), FloatSlider(value=0.1, max=1.0, step…

In [62]:
np.unique(pred_masks)

array([0], dtype=int64)